In [1]:
!pip install transformers datasets scikit-learn --quiet

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/arabic-empathetic-conversations.csv')

In [4]:
df.head()

,emotion,context,response
0,sentimental,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,sentimental,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,sentimental,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,afraid,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,afraid,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [5]:
df.shape

(36628, 3)

In [6]:
df = df.drop('emotion', axis=1)
df.head()

,context,response
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [7]:
df = df.rename(columns={
    'context': 'Question',
    'response': 'Answer',
})

df.head()

,Question,Answer
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [8]:
import re
def clean_text(text):
  """
  Removes punctuation, numbers, and dates from the text.
  """
  text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
  text = re.sub(r'\d+', '', text)  # Remove numbers
  text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)  # Remove dates (DD/MM/YYYY)
  return text


# Clean questions and answers
df['Question'] = df['Question'].astype(str).apply(clean_text)
df['Answer'] = df['Answer'].astype(str).apply(clean_text)

df.head()


,Question,Answer
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقا كنت تحبه أم مجرد أفضل صديق
1,كان هذا أفضل صديق اشتقت لها,اين ذهبت
2,لم نعد نتحدث,هل كان هذا شيء حدث بسبب جدال
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل أنا حقا لا أرى كيف
4,ألا تشعر بذلك إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [9]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# تحويل البيانات إلى قوائم
train_questions = train_df['Question'].tolist()
train_answers = train_df['Answer'].tolist()

test_questions = test_df['Question'].tolist()
test_answers = test_df['Answer'].tolist()

start exp 2

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader, Dataset

# تحديد الجهاز
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# تحميل النموذج المسبق التدريب
model_name = "malmarjeh/bert2bert"  # يمكنك اختيار نموذج آخر يدعم العربية
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)  # نقل النموذج إلى الجهاز

# تحضير البيانات
train_encodings = tokenizer(train_questions, truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(test_questions, truncation=True, padding=True, return_tensors="pt")

# إضافة الإجابات كـ 'labels'
train_encodings['labels'] = tokenizer(train_answers, truncation=True, padding=True, return_tensors="pt").input_ids
test_encodings['labels'] = tokenizer(test_answers, truncation=True, padding=True, return_tensors="pt").input_ids

# فئة Dataset
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = QADataset(train_encodings)
test_dataset = QADataset(test_encodings)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [15]:
import time

# التدريب
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):  # عدد العهود
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Epoch {epoch + 1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")

Epoch 1, Batch 0/3663, Loss: 18.270225524902344
Epoch 1, Batch 10/3663, Loss: 1.0013530254364014
Epoch 1, Batch 20/3663, Loss: 0.7473078966140747
Epoch 1, Batch 30/3663, Loss: 0.47864818572998047
Epoch 1, Batch 40/3663, Loss: 0.537832498550415
Epoch 1, Batch 50/3663, Loss: 0.5620681047439575
Epoch 1, Batch 60/3663, Loss: 0.636870801448822
Epoch 1, Batch 70/3663, Loss: 0.46869900822639465
Epoch 1, Batch 80/3663, Loss: 0.4957076609134674
Epoch 1, Batch 90/3663, Loss: 0.4154857397079468
Epoch 1, Batch 100/3663, Loss: 0.4923975467681885
Epoch 1, Batch 110/3663, Loss: 0.37491264939308167
Epoch 1, Batch 120/3663, Loss: 0.42975759506225586
Epoch 1, Batch 130/3663, Loss: 0.4357912838459015
Epoch 1, Batch 140/3663, Loss: 0.47789114713668823
Epoch 1, Batch 150/3663, Loss: 0.6039135456085205
Epoch 1, Batch 160/3663, Loss: 0.5292586088180542
Epoch 1, Batch 170/3663, Loss: 0.45902925729751587
Epoch 1, Batch 180/3663, Loss: 0.6036068797111511
Epoch 1, Batch 190/3663, Loss: 0.500870943069458
Epoch 1,

In [16]:
# التقييم
model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)  # نقل إلى الجهاز
        attention_mask = batch['attention_mask'].to(device)  # نقل إلى الجهاز
        labels = batch['labels'].to(device)  # نقل إلى الجهاز

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        print(f"Test Loss: {loss.item()}")


Test Loss: 0.3404100239276886
Test Loss: 0.4940689504146576
Test Loss: 0.5646886825561523
Test Loss: 0.6226295828819275
Test Loss: 0.751798689365387
Test Loss: 0.4322951138019562
Test Loss: 0.45515236258506775
Test Loss: 0.4662712514400482
Test Loss: 0.472938597202301
Test Loss: 0.4007164537906647
Test Loss: 0.7487051486968994
Test Loss: 0.5540327429771423
Test Loss: 0.5782809853553772
Test Loss: 0.647214949131012
Test Loss: 0.4343603849411011
Test Loss: 0.5288272500038147
Test Loss: 0.7251087427139282
Test Loss: 0.695250391960144
Test Loss: 0.6432098746299744
Test Loss: 0.5896689295768738
Test Loss: 0.35161733627319336
Test Loss: 0.41548094153404236
Test Loss: 0.5764836072921753
Test Loss: 0.4674030840396881
Test Loss: 0.462610125541687
Test Loss: 0.8026758432388306
Test Loss: 0.6786308288574219
Test Loss: 0.3592184782028198
Test Loss: 0.5369258522987366
Test Loss: 0.42173945903778076
Test Loss: 0.6447480916976929
Test Loss: 0.5267817974090576
Test Loss: 0.48163682222366333
Test Loss:

In [17]:
# التنبؤ
def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt").to(device)  # نقل إلى الجهاز
    outputs = model.generate(inputs['input_ids'], max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# مثال استخدام
question = "انا اليوم اشعر بشعور كئيب جدا "
answer = generate_answer(question)
print(answer)

أنا آسف لسماع ذلك أنا آسف لسماع ذلك


end